In [ ]:
!pip3 install numpy
!pip3 install pandas
!pip3 install scipy
!pip3 install matplotlib

In [5]:
import pandas as pd

In [32]:
df1 = pd.read_csv('../processed_data/Ireland_Data.csv', encoding="utf8")
df2 = pd.read_csv('../processed_data/Fayetville_Data.csv', encoding="utf8")
df3 = pd.read_csv('../processed_data/Post-Fayetville_Data.csv', encoding="utf8")

In [33]:
dataframes = [df1, df2, df3]

In [34]:
combined = pd.concat(dataframes)
combined = combined.drop(['Unnamed: 0'], axis=1)

In [35]:
combined

,Date,Exercise,Weights,Sets,Reps,RPE,Notes,Mobility
0,5/31/19,dips,[45.0],[3],[8],[None],very difficult rpe10 every set,NaN
1,6/2/19,tng bench,[185.0],[3],[10],[None],rpe6 sleeved without much leg drive and qualit...,NaN
2,6/2/19,beltless high bar,[275.0],[3],[8],[None],"feels very good considering sleep schedule, rpe8",NaN
3,6/3/19,comp bench,[195.0],[3],[10],[None],rpe8 good and smooth,NaN
4,6/3/19,pause beltless sumo deadlift,"[425.0, 375.0]","[1, 4]","[1, 5]","[None, None]",smooth but heavy,NaN
...,...,...,...,...,...,...,...,...
101,8/3/20,comp bench,"[295.0, 255.0]","[1, 4]","[1, 5]","[7.0, 7.0]",either only have primer singles the day before...,"mobility, warmup"
102,8/5/20,comp deadlift,[425.0],[3],[7],[None],fucking tiring,"56cal 10min bike ride warmup, mobility, warmup"
103,8/5/20,spoto press,[245.0],[4],[3],[None],money and no tendonitis,"56cal 10min bike ride warmup, mobility, warmup"
104,8/7/20,pause comp squat,[425.0],[1],[1],[7.5],"8, 365x3x3 / flats for first time and feel great","58cal 10min bike ride warmup, mobility, warmup"


In [49]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.dates import date2num

# Assuming df is your DataFrame and it has been populated as per previous examples

# Filter the DataFrame for 'bench' related exercises, assuming all variations contain "comp bench"
ex1 = combined[combined['Exercise'].str.contains('comp bench', case=False, na=False)]

# Prepare lists to store plot data
dates = []
weights = []
reps = []


# Iterate over the filtered DataFrame
for index, row in ex1.iterrows():
    # Check each weight and corresponding rep count
    for weight, rep in zip(row['Weights'], row['Reps']):
        if row['Reps'] > 0 and row['Weights'] > 150:  # Only consider reps greater than zero
            dates.append(row['Date'])
            weights.append(row['Weights'] )
            reps.append(row['Reps'])

# Convert dates for plotting
dates = pd.to_datetime(dates)
# print('after', inclineDates)
date_nums = date2num(dates)  # Convert dates to ordinal numbers for regression

# Use a colormap to vary colors based on the number of reps
cmap = plt.get_cmap('viridis')
norm = plt.Normalize(min(reps), max(reps))

# Compute linear regression
slope, intercept = np.polyfit(date_nums, weights, 1)
regression_line = slope * date_nums + intercept

# Plotting
plt.figure(figsize=(16, 12))
scatter = plt.scatter(dates, weights, c=reps, cmap=cmap, norm=norm, s=100, edgecolor='k')
plt.plot(dates, regression_line, color='black', linewidth=2, label='Weight Regression')  # Add regression line
plt.scatter(dates, weights, c=reps, cmap=cmap, norm=norm, s=100, edgecolor='k', marker='+')
plt.title('All Bench Sets Over Time')
plt.xlabel('Date')
plt.ylabel('Weight (lbs)')
plt.colorbar(scatter, label='Number of Reps')
plt.grid(True)
plt.xticks(rotation=45)  # Rotate date labels for better readability
plt.legend()
plt.tight_layout()  # Adjust layout to make room for label rotation
plt.show()

TypeError: '>' not supported between instances of 'str' and 'int'

In [36]:
combined.to_csv("../processed_data/Combined_Data.csv")